## Finding Hassi Messaoud plume from Varon (2021) paper (Hassi Messaoud plume)
file:///C:/Users/s1709837/OneDrive%20-%20University%20of%20Edinburgh/Documents/Papers%20and%20Reports/varon_2021.pdf

Source: 31.6585◦ N, 5.9053◦ E
Date: 20/11/2019

ImageCollection package help:
https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate

In [26]:
import os

import ee
ee.Authenticate()

#Initialize earth engine for a new Python session

ee.Initialize(project = 'ee-krgeophys')

import geemap

from geemap import *

import numpy as np
import xarray as xr # May need to install using %pip install xarray
import datetime

from matplotlib import pyplot as plt

### Hassi Messaoud plot Set up

#### Define date and location of plume source

In [27]:
# Define date and location of Hassi Messaoud plume

source_lat = 31.6585
source_lon = 5.9053
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2019-11-20'
plume_date_end = '2019-11-21'
print('Plume detection date:' , plume_date_start)

# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = 5.88419
lonmax = 5.92609
latmin = 31.64011
latmax = 31.67640

# Define GEE geometry shape over this area
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

Plume source at ( 31.6585 , 5.9053 )
Plume detection date: 2019-11-20


#### Load image for S2 L-1C (TOA) Hassi Messaoud on 20/11/2019

In [28]:
# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()

# Print image ID and spacecraft name
print('Image ID:', img.id().getInfo())
print('Spacecraft name:', img.get('SPACECRAFT_NAME').getInfo())

Image ID: 20191120T101321_20191120T101408_T31SGR
Spacecraft name: Sentinel-2A


#### Plot Band 12 TOA reflectance over Hassi Massaoud

In [29]:
from matplotlib import pyplot as plt

In [30]:
# Plot only B12 to see plume as show in Varon (2021) Figure 2b
mapB12 = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 0.35,
    'max': 0.75,   
    'bands': ['B12']
   
}

mapB12.setCenter(source_lon, source_lat, 13.5) # Define map centre
mapB12.addLayer(img.divide(10000), vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
# Map.remove_colorbar(clear)
mapB12.add_colorbar(vis, label='TOA Refelctance')

mapB12

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [31]:
# Plot only B12 to see plume as show in Varon (2021) Figure 2b
mapB11 = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 0.35,
    'max': 0.75,   
    'bands': ['B11']
   
}

mapB11.setCenter(source_lon, source_lat, 13.5) # Define map centre
mapB11.addLayer(img.divide(10000), vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
# Map.remove_colorbar(clear)
mapB11.add_colorbar(vis, label='TOA Refelctance')

mapB11

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Plot Hassi Messaoud MBSP enhancememnt image

#### Compute MBSP methane enhancement

In [32]:
# Linear fit for a region, applying reduceRegion to ee.Image as laid out by https://developers.google.com/earth-engine/guides/reducers_regression

# Define geometry
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Subset the SWIR1 and SWIR2 bands. In the regression reducer, independent
 # variables come first followed by the dependent variables. In this case,
 # B5 (SWIR1) is the independent variable and B6 (SWIR2) is the dependent
 # variable.
imRegress = img.select(['B11', 'B12'])

# Calculate the regressions coefficiets for the set of pixels intersecting the above defined region using reduceRegion with ee.Reducer.linearFit()


linearFit = imRegress.reduceRegion(
    reducer = ee.Reducer.linearFit(),
    geometry = geometry_area,
   
)                          

linearFit



### Calculate linear regression with offset forced through 0

In [33]:
# Import image using above
img = ee.Image('COPERNICUS/S2_HARMONIZED/20191120T101321_20191120T101408_T31SGR')

# Create new image that is the concatenation of three images: a constant, the SWIR1 band and the SWIR2 band
B11 = img.select('B11').divide(10000)
B12 = img.select('B12').divide(10000)
imRegress = ee.Image.cat(B11, B12)

linearRegression = imRegress.reduceRegion(
    reducer = ee.Reducer.linearRegression(numX = 1, numY = 1),
    geometry = geometry_area,
    scale = 20,
)

linearRegression.get('coefficients').getInfo()[0]


[0.8682509285575256]

### Plot MBSP enhancement with plume

#### MBSP plot with optimised colorbar

In [34]:
# Plot the enhancement found with empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Define scaling factor c from linear regression
c = linearRegression.get('coefficients').getInfo()[0]

In [35]:
c

[0.8682509285575256]

In [36]:
# Use image.expression() to write equation for R_MBSP
R_MBSP = img.expression(
    '(c*R12 - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        
        # 'c': linearRegression.get('coefficients').getInfo()[0],
        'c': c, 
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement in ppm
F_MBSP = R_MBSP.subtract(-0.029)

In [37]:
# Get max and min R_MBSP values to compare with Fei's (min - -0.16 and max = -0.0150)
minval = F_MBSP.reduceRegion(ee.Reducer.min(), geometry = geometry_area).get('constant').getInfo()
maxval = F_MBSP.reduceRegion(ee.Reducer.max(), geometry = geometry_area).get('constant').getInfo()

print('min_val:',F_MBSP.reduceRegion(ee.Reducer.min(), geometry = geometry_area).get('constant').getInfo())
print('max_val:',F_MBSP.reduceRegion(ee.Reducer.max(), geometry = geometry_area).get('constant').getInfo())

min_val: -0.29751576416396047
max_val: -0.1613932033663398


In [38]:
# Display F_MBSP on map

Map = geemap.Map()

# Define visualisation parameters for plot
# vis = { 'min': -0.5, 'max': 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
# vis = { 'min': -2, 'max': 2, "palette": ['FF0000',	'FFFFFF', '0000FF']}
vis = { 'min': minval, 'max': maxval, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### For Radiative transfer code - get array of these mbsp values

In [39]:
# Define date and location of Hassi Messaoud plume

source_lat = 31.6585
source_lon = 5.9053
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2019-11-20'
plume_date_end = '2019-11-21'
print('Plume detection date:' , plume_date_start)


#### Make a rectangle over area over which Varon 2021 images the Hassi Messaoud plume

# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = source_lon - 0.008
lonmax = source_lon + 0.008
latmin = source_lat - 0.008
latmax = source_lat + 0.008

# Set up GEE geometry shape
small_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)


Plume source at ( 31.6585 , 5.9053 )
Plume detection date: 2019-11-20


In [40]:
F_small_area = F_MBSP.sampleRectangle(small_area)

In [41]:
# Turn list into 2d array and check its shape
F_array = np.array((F_small_area.get('constant').getInfo()))
F_array.shape

(91, 79)

#### Run retrieval from Varon demonstration

In [42]:
import setup
import radtran as rt

In [43]:
pip install time

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [44]:
import time
import tqdm


In [45]:
F_array.size

7189

In [46]:
# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = source_lon - 0.0002
lonmax = source_lon + 0.0002
latmin = source_lat - 0.0002
latmax = source_lat + 0.0002

# Set up GEE geometry shape
tiny_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)


In [20]:
# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = source_lon - 0.00005
lonmax = source_lon + 0.00005
latmin = source_lat - 0.00005
latmax = source_lat + 0.00005

# Set up GEE geometry shape
tiny_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)


In [22]:
F_tiny_area = F_MBSP.sampleRectangle(tiny_area)

In [23]:
# Turn list into 2d array and check its shape
F_tiny_array = np.array((F_tiny_area.get('constant').getInfo()))
F_tiny_array.shape

(1, 1)

In [24]:
# Configuration
num_layers = 100
targheight = 0
obsheight = 100
solarangle = 40
obsangle = 0
instrument = 'S2A'
method = 'MBSP'

# Invented 2D array of fractional reflectance data
frac_refl_data = F_tiny_array




In [47]:
test_retrieval = rt.retrieve(np.array([[-0.2]]), instrument, method, targheight, obsheight, solarangle, obsangle, num_layers=num_layers)

Creating the transmission spectrum...
--- 7.133105516433716 seconds --- to interpolate
--- 12.43881106376648 seconds --- to run radtran()
Creating the transmission spectrum...
--- 21.696653842926025 seconds --- to interpolate
--- 27.897725105285645 seconds --- to run radtran()
--- 0.0010919570922851562 seconds --- to optimize
1


In [48]:
#Takes too long, set up below for a smaller area
from tqdm import tqdm
import time
for i in tqdm(range(F_tiny_array.size), desc = 'tqdm() Progress Bar'):
    test_retrieval = rt.retrieve(frac_refl_data, instrument, method, targheight, obsheight, solarangle, obsangle, num_layers=num_layers)
    time.sleep(0.5)

tqdm() Progress Bar:   0%|          | 0/1 [00:00<?, ?it/s]

Creating the transmission spectrum...
--- 7.536694765090942 seconds --- to interpolate
--- 12.31460189819336 seconds --- to run radtran()
Creating the transmission spectrum...
--- 15.02354645729065 seconds --- to interpolate
--- 20.902823448181152 seconds --- to run radtran()
--- 0.0 seconds --- to optimize
1


tqdm() Progress Bar: 100%|██████████| 1/1 [00:33<00:00, 33.74s/it]


In [55]:
test_retrieval # units mol/m2

array([[ 7.46606612,  8.79593243, 10.31568551,  8.07673679,  6.46124706],
       [ 7.28359862,  7.93455732,  9.51711693,  8.0919969 ,  6.08532714],
       [ 6.82017208,  6.86402707,  7.84257516,  7.16340729,  6.18825591],
       [ 6.24237441,  6.32080893,  6.96132545,  6.43719434,  6.09818444],
       [ 6.30486065,  6.29437473,  6.46084264,  6.08491073,  5.82450991]])

In [56]:
#### For non plume pass of same area, see later on

#### MBSP plot with Varon 2021 colorbar

In [20]:
# Plot basemap
Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP

# Values of c - 0.8682509285575336 for b12 over b11 with intercept = 0; 1.1512279743749667 for B11 over B12 with y=0
R_MBSP = img.expression(
    '(0.8682509285575336*R12 - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement in ppm
F_MBSP = R_MBSP.subtract(-0.029)

# Define visualisation parameters for plot
# vis = { 'min': -0.38, 'max': 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
vis = { 'min': -2, 'max': 2, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### MBSP converted to mol/m2, assuming initial map from ratio is in ppm - doesn't give expected result

In [12]:
# Convert from ppm to mol/m2

# Define variables
dry_air_col = 10000 # Dry air column in kg/m2

# Get CH4 fraction from ppm value
ch4_fraction = F_MBSP.divide(1000000)

# Get CH4 column value by multiplying by dry are column
ch4_col = ch4_fraction.multiply(dry_air_col)

In [13]:
# Convert to g/m2
ch4_col_grams = ch4_col.multiply(1000)

# Convert from mass to moles (m. mass of ch4 = 16g/mol)
ch4_conc_enh = ch4_col_grams.divide(16)

In [14]:
ch4_col.reduceRegion(reducer = ee.Reducer.min(), geometry = geometry_area).get('constant')

In [15]:
ch4_conc_enh.reduceRegion(reducer = ee.Reducer.min(), geometry = geometry_area).get('constant')

In [16]:
F_MBSP.reduceRegion(reducer = ee.Reducer.min(), geometry = geometry_area)

In [17]:
Map = geemap.Map()

# vis = { 'min': -0.38, 'max': 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
vis = { 'min': -2, 'max': 2, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(ch4_conc_enh, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Compute MBMP

enh(MBMP) = enh(MBSP) - enh'(MBSP)

So we compute MBSP retrieval for a day with no plume (Use 6/10/2019 as in Varon 2021), and subtract this from the MBSP image for the 20/11/2019

#### Compute MBSP for 6/10/2019

In [62]:
# Define date and location of Hassi Messaoud plume

source_lat = 31.6585
source_lon = 5.9053
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2019-10-06'
plume_date_end = '2019-10-07'
print('Plume detection date:' , plume_date_start)


#### Make a rectangle over area over which Varon 2021 images the Hassi Messaoud plume

# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
lonmin = 5.88419
lonmax = 5.92609
latmin = 31.64011
latmax = 31.67640

# Set up GEE geometry shape
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

Plume source at ( 31.6585 , 5.9053 )
Plume detection date: 2019-10-06


In [63]:
# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

In [64]:
# Find out whether it's Sentinel 2A or 2B
# List of image properties: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#image-properties
img.get('SPACECRAFT_NAME')

# Therefore, model m should be  mMBSP = −0.029 for Sentinel-2A
# For Sentinel-2B, m = -0.022

#### Plot Band 12 TOA reflectance over Hassi Massaoud on day with no plume

In [65]:
# Plot only B12 to see that there is no plume on 6/10/2019
mapB12 = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 0.35,
    'max': 0.75,   
    'bands': ['B12']
   
}

mapB12.setCenter(source_lon, source_lat, 13.5) # Define map centre
mapB12.addLayer(img.divide(10000), vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
# Map.remove_colorbar(clear)
mapB12.add_colorbar(vis, label='TOA Refelctance')

mapB12

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Compute linear regression of B12 over B11 for day without plume

In [66]:
# Create new image that is the concatenation of three images: a constant, the SWIR1 band and the SWIR2 band
xVar = img.select('B11')
yVar = img.select('B12')
imRegress = ee.Image.cat(xVar, yVar)

# Compute linear regression
linearRegression = imRegress.reduceRegion(
    reducer = ee.Reducer.linearRegression(numX = 1, numY = 1),
    geometry = geometry_area,
    scale = 20,
)

linearRegression.get('coefficients')

### Plot MBSP without plume

In [68]:
# Plot the enhancement found without empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP_np (R_MBSP with no plume (np))
R_MBSP_np = img.expression(
    '(0.8759245145609675*R12 - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement for day with no plume, subtracting 0/022 for sentinel-b
F_MBSP_np = R_MBSP_np.subtract(-0.022)

# Define visualisation parameters for plot
vis = { 'min': -0.3, 'max': 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
# vis = { 'min': -2.0, 'max': 2.0, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(F_MBSP_np, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [69]:
F_tiny_area = F_MBSP_np.sampleRectangle(tiny_area)

In [70]:
# Turn list into 2d array and check its shape
F_tiny_array = np.array((F_tiny_area.get('constant').getInfo()))
F_tiny_array.shape

(5, 5)

In [71]:
# Configuration
num_layers = 100
targheight = 0
obsheight = 100
solarangle = 40
obsangle = 0
instrument = 'S2A'
method = 'SBMP'

# Invented 2D array of fractional reflectance data
frac_refl_data = F_tiny_array




In [72]:
#Takes too long, set up below for a smaller area
from tqdm import tqdm
import time
for i in tqdm(range(F_tiny_array.size), desc = 'tqdm() Progress Bar'):
    test_retrieval = rt.retrieve(frac_refl_data, instrument, method, targheight, obsheight, solarangle, obsangle, num_layers=num_layers)
    time.sleep(0.5)

tqdm() Progress Bar:   0%|                                                                      | 0/25 [00:00<?, ?it/s]

Creating the transmission spectrum...
--- 18.349987506866455 seconds --- to run radtran()
--- 0.012280702590942383 seconds --- to optimize


tqdm() Progress Bar:   4%|██▍                                                           | 1/25 [00:18<07:32, 18.87s/it]

Creating the transmission spectrum...
--- 19.09066390991211 seconds --- to run radtran()
--- 0.012082576751708984 seconds --- to optimize


tqdm() Progress Bar:   8%|████▉                                                         | 2/25 [00:38<07:24, 19.31s/it]

...

tqdm() Progress Bar:  92%|████████████████████████████████████████████████████████     | 23/25 [07:01<00:35, 17.75s/it]

Creating the transmission spectrum...
--- 17.256232500076294 seconds --- to run radtran()
--- 0.009000778198242188 seconds --- to optimize


tqdm() Progress Bar:  96%|██████████████████████████████████████████████████████████▌  | 24/25 [07:18<00:17, 17.76s/it]

Creating the transmission spectrum...
--- 17.206979513168335 seconds --- to run radtran()
--- 0.008999824523925781 seconds --- to optimize


tqdm() Progress Bar: 100%|█████████████████████████████████████████████████████████████| 25/25 [07:36<00:00, 18.27s/it]


In [73]:
test_retrieval

array([[6.69796722, 6.69070202, 6.46855894, 6.02573654, 5.99546463],
       [6.75126743, 6.67526815, 6.30226961, 5.70534348, 5.84860368],
       [6.56312243, 6.40218193, 5.75318104, 5.55186877, 5.59892141],
       [6.23102373, 6.03162934, 5.4975979 , 5.67146012, 5.61037451],
       [6.06619968, 5.78689889, 5.58110371, 5.28878799, 5.13768575]])

### Plot MBMP (MBSP - MBSP_np)

In [31]:
F_MBMP = F_MBSP.subtract(F_MBSP_np)

In [32]:
# Plot the enhancement found without empirical scaling factor c (least squares fit of B12 against B11)
# Plot basemap
Map = geemap.Map() 

# Get MBMP enhancement using temporal normalisation - subtract single pass image of non-plume day from single pass image of plume day
enhancement_MBMP = F_MBSP.subtract(F_MBSP_np)

# Define visualisation parameters for plot
# vis = {'min': -0.2, 'max' : 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
vis = { 'min': -2.0, 'max': 2.0, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(enhancement_MBMP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Clip MBMP image to 10x10km region

In [33]:
Map = geemap.Map()

# Clip MBMP image to the geometry area of fig 2f in Varon 2021
Hassi_Messaoud_area = enhancement_MBMP.clip(geometry_area)

vis = {'min': -2.0, 'max' : 2.0, "palette": ['FF0000',	'FFFFFF', '0000FF']}
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre
Map.addLayer(Hassi_Messaoud_area, vis, 'Clipped Image')
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [34]:
#Define region of interest as plume polygon drawn above
roi_plume = ee.FeatureCollection(Map.draw_features)
roi_plume

In [35]:
roi_plume_geom = roi_plume.geometry(maxError=1)
roi_plume_geom

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Collection.geometry",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection",
          "arguments": {
            "features": {
              "constantValue": []
            }
          }
        }
      },
      "maxError": {
        "functionInvocationValue": {
          "functionName": "ErrorMargin",
          "arguments": {
            "value": {
              "constantValue": 1
            }
          }
        }
      }
    }
  }
})

### Get IME and Source Rate Q of Hassi Messaoud plume 20/11/19

In [36]:
# Get median of values in this polygon using ee.Image.reduceRegion
plume_enhancement_median = ee.Image.reduceRegion(image = Hassi_Messaoud_area, reducer = ee.Reducer.median(), geometry = roi_plume, scale = 20)

#Calculate sum of plume enhancement (~ -7.65 according to one go... take the absolute of this since everything is negative)
plume_enhancement_sum  = ee.Image.reduceRegion(image = Hassi_Messaoud_area, reducer = ee.Reducer.sum(), geometry = roi_plume, scale = 20)
plume_enhancement_sum.getInfo()

# Get enhancement map of just plume by clipping to its geometry
roi_plume_geom = roi_plume.geometry()
plume_only_map = enhancement_MBMP.clip(roi_plume_geom)

# Convert plume map values from mol/m2 to kg/m2
plume_values_kgm2 = plume_only_map.multiply(16.4/1000)

# Get IME for each pixel and across whole area
IME_pixel = plume_values_kgm2.multiply(400)

# Get plume IME - Calculate sum of plume enhancement 
IME  = abs(ee.Image.reduceRegion(image = IME_pixel, reducer = ee.Reducer.sum(), geometry = roi_plume, scale = 20).get('constant').getInfo())

# Get plume area in m^2 as a float
plume_area = roi_plume_geom.area(maxError=1).getInfo()

# Get plume length as root of area
L = np.sqrt(plume_area)

# Q in kg/s = (IME*\ueff)/l 
Q_kgs = (IME * 10)/L

# Q in t/hour (3600s in 1 hour, 1000kg in 1t)
Q_th = Q_kgs*3600/1000

print('IME :', IME, ' \nQ in kg/s:', Q_kgs, '\nQ in t/hour:', Q_th )

EEException: Image.clip: The geometry for image clipping must not be empty.

## Get Hassi Messaoud source rate using Varon's method of 3x3 median filter and 95th percentile to delineate plume
"Here we define the plume mask by selecting methane columns above the 95th percentile for the scene and smooth with a 3 × 3 median filter." (Varon et al., 2021)

In [125]:
from pprint import pprint

uniformKernel = (
    ee.Kernel.square(
        radius =  1,
        units =  'pixels',
        magnitude = 1,
    )
)

In [128]:
pprint(uniformKernel.getInfo())

{'center': [1, 1],
 'radius': 1,
 'type': 'Kernel.square',
 'weights': '\n'
            '  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]\n'
            '  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]\n'
            '  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111]'}


In [129]:
# Plot Hassi Messaoud MBMP 10x10 area clipped in previous section
Map = geemap.Map()


vis = {'min': -0.2, 'max' : 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre
Map.addLayer(Hassi_Messaoud_area, vis, 'Clipped Image')
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [130]:
# Convolve image with kernel

filtered_img = Hassi_Messaoud_area.convolve(uniformKernel)


In [131]:
Map = geemap.Map()


vis = {'min': -0.2, 'max' : 0.05, "palette": ['FF0000',	'FFFFFF', '0000FF']}
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre
Map.addLayer(filtered_img, vis, 'Clipped Image')
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

## Kazakhstan blowout plume: applying above methodolody (From Varon et al. (2021)) to search for Kazakhstan plume
https://eartharxiv.org/repository/view/6709/ (Guanter 2023 preprint)
Using sentinel-2 retrieval data in Table S1 (Guanter 2023 preprint)

Source (location of blowout specified in Guanter (2023 preprint) Fig 1a:  (45.3324◦N, 52.3730◦E)
Date: 09/06/2023

ImageCollection package help:
https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate

In [38]:
import os

import ee
ee.Authenticate()

#Initialize earth engine for a new Python session

ee.Initialize()

import geemap

from geemap import *

import numpy as np
import xarray as xr # May need to install using %pip install xarray
import datetime

from matplotlib import pyplot as plt

from datetime import datetime as dt
from datetime import timedelta

#### Set up plot, visualise B12 over relevant area and date

#### Define date and location of plume source

In [39]:
# Define date and location of Hassi Messaoud plume

source_lat = 45.3324
source_lon = 52.3730
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

plume_date_start = '2023-06-09'
plume_date_end = '2023-06-10'
print('Plume detection date:' , plume_date_start)


Plume source at ( 45.3324 , 52.373 )
Plume detection date: 2023-06-09


#### Make a rectangle over ~10x10km area 

In [40]:
# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
# BLcorner = 45.276069, 52.292236
#URcorner = 45.374880, 52.435128
lonmin = 52.292236
lonmax = 52.435128
latmin = 45.276069
latmax = 45.374880

# Set up GEE geometry shape
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

#### Plot Sentinel-2 data over this area for day of plume

#### Load image for S2 L-1C (TOA) Hassi Messaoud on 20/11/2019

In [41]:
# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

In [42]:
# Find out whether it's Sentinel 2A or 2B
# List of image properties: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#image-properties
img.get('SPACECRAFT_NAME')

# Therefore, model m should be  mMBSP = −0.022 for Sentinel-2B - MAYBE THIS HAS TO CHANGE FOR THE DIFFERENT OBSERVATION?

#### Plot B12 only

In [43]:
# Plot only B12 to see plume as show in Varon (2021) Figure 2b
Map = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 2000,
    'max': 4000,   
    'bands': ['B12'],
    # "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"]
   
}

Map.setCenter(source_lon, source_lat, 14) # Define map centre
Map.addLayer(img, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Least squares fitting (linear fit) https://developers.google.com/earth-engine/guides/reducers_regression#linearfit_2

In [44]:
# Linear fit for a region, applying reduceRegion to ee.Image as laid out by https://developers.google.com/earth-engine/guides/reducers_regression

# Define geometry
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Subset the SWIR1 and SWIR2 bands. In the regression reducer, independent
 # variables come first followed by the dependent variables. In this case,
 # B5 (SWIR1) is the independent variable and B6 (SWIR2) is the dependent
 # variable.
imRegress = img.select(['B11', 'B12'])

# Calculate the regressions coefficiets for the set of pixels intersecting the above defined region using reduceRegion with ee.Reducer.linearFit()


linearFit = imRegress.reduceRegion(
    reducer = ee.Reducer.linearFit(),
    geometry = geometry_area,   
)                          

linearFit


### Image operation

In [45]:
# Plot the enhancement found with empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP
R_MBSP = img.expression(
    '(( 0.9714300514220611*R12) - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.022)

# Define visualisation parameters for plot
vis = { 'min': -0.5, 'max': 0.1, "palette": ['FF0000',	'FFFFFF', '0000FF']}


Map.setCenter(source_lon, source_lat, 13.) # Define map centre
Map.addLayer(F_MBSP, vis,  'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Kazakhstan MBMP

In [46]:
# Define date and location of Hassi Messaoud plume

source_lat = 45.3324
source_lon = 52.3730
print('Plume source at (', source_lat, ',',source_lon, ')')

# Define dates to filter image collection by (date of plume detection)

# plume_date_start = '2023-06-09'
# plume_date_end = '2023-06-10'

plume_date_start = '2023-06-04'
plume_date_end = '2023-06-05'

print('Plume detection date:' , plume_date_start)


# Define geometry as rectangle with coordinates at 2km distance from source (found using google maps)
# BLcorner = 45.276069, 52.292236
#URcorner = 45.374880, 52.435128
lonmin = 52.292236
lonmax = 52.435128
latmin = 45.276069
latmax = 45.374880

# Set up GEE geometry shape
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

#### Plot Band 12 TOA reflectance over Kazakhstan site on day with no plume

# Load TOA collection
collection = (
    ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
    .filterBounds(geometry_area) # Filter by geographical area using geometry rectangle defined above
    .filterDate(plume_date_start, plume_date_end) # Filter by date of plume detection
)

# Get image as first image for filtered collection
img = collection.first()
img

# Therefore, model m should be  mMBSP = −0.022 for Sentinel-2B - MAYBE THIS HAS TO CHANGE FOR THE DIFFERENT OBSERVATION?

# Plot only B12 to see plume as show in Varon (2021) Figure 2b
Map = geemap.Map()
# Define visualisation parameters
vis = {
    'min': 2000,
    'max': 4000,   
    'bands': ['B12'],
    # "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"]
   
}

Map.setCenter(source_lon, source_lat, 14) # Define map centre
Map.addLayer(img, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Plume source at ( 45.3324 , 52.373 )
Plume detection date: 2023-06-04


Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [47]:
# Find out whether it's Sentinel 2A or 2B
# List of image properties: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_HARMONIZED#image-properties
img.get('SPACECRAFT_NAME')

In [48]:
# Linear fit for a region, applying reduceRegion to ee.Image as laid out by https://developers.google.com/earth-engine/guides/reducers_regression

# Define geometry
geometry_area = ee.Geometry.Rectangle([lonmin, latmin, lonmax, latmax], None, False)

# Subset the SWIR1 and SWIR2 bands. In the regression reducer, independent
 # variables come first followed by the dependent variables. In this case,
 # B5 (SWIR1) is the independent variable and B6 (SWIR2) is the dependent
 # variable.
imRegress = img.select(['B11', 'B12'])

# Calculate the regressions coefficiets for the set of pixels intersecting the above defined region using reduceRegion with ee.Reducer.linearFit()


linearFit = imRegress.reduceRegion(
    reducer = ee.Reducer.linearFit(),
    geometry = geometry_area,   
)                          

linearFit


In [49]:
pip install jupyterlab-widgets==1.1.1
jupyter-labextension list
JupyterLab v3.4.5
/opt/anaconda/envs/ipyw/share/jupyter/labextensions
        jupyterlab_pygments v0.2.2 enabled OK (python, jupyterlab_pygments)
        @jupyter-widgets/jupyterlab-manager v3.1.1 enabled OK (python, jupyterlab_widgets)

SyntaxError: invalid syntax (2793911198.py, line 1)

In [50]:
# Plot the enhancement found with empirical scaling factor c (least squares fit of B12 against B11)

from matplotlib import pyplot as plt

Map = geemap.Map() # plot basemap

# Use image.expression() to write equation for R_MBSP
R_MBSP = img.expression(
    '(( 0.8964624786212904*R12) - R11)/R11 ', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
    },
)

# Methane column enhancement
F_MBSP_np = R_MBSP.subtract(-0.022)

# Define visualisation parameters for plot
vis = { 'min': -0.5, 'max': 0.1, "palette": ['FF0000',	'FFFFFF', '0000FF']}


Map.setCenter(source_lon, source_lat, 13.) # Define map centre
Map.addLayer(F_MBSP_np, vis,  'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [71]:
from PIL import Image

img1 = Image.open('dem.tif', 'r')
img1.show()

FileNotFoundError: [Errno 2] No such file or directory: 'dem.tif'

In [51]:
F_MBMP = F_MBSP.subtract(F_MBSP_np)

In [52]:
# Plot the enhancement found without empirical scaling factor c (least squares fit of B12 against B11)
# Plot basemap
Map = geemap.Map() 

# Get MBMP enhancement using temporal normalisation - subtract single pass image of non-plume day from single pass image of plume day
enhancement_MBMP = F_MBSP.subtract(F_MBSP_np)

# Define visualisation parameters for plot
vis = {'min': -0.2, 'max' : 0.2, "palette": ['FF0000',	'FFFFFF', '0000FF']}

Map.setCenter(source_lon, source_lat, 13.75) # Define map centre
Map.addLayer(enhancement_MBMP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

#### Clip this MBMP enhancement image to 10x10km




In [56]:
Map = geemap.Map()

# Clip MBMP image to the geometry area of fig 2f in Varon 2021
area_10km = enhancement_MBMP.clip(geometry_area)

vis = {'min': -0.2, 'max' : 0.2, "palette": ['FF0000',	'FFFFFF', '0000FF']}
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre
Map.addLayer(area_10km, vis, 'Clipped Image')
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### Calculate IME and source rate

#### Cut out plume and calculate median and sum of enhancemnt inside

In [57]:
#Define region of interest as plume polygon drawn above
roi_plume = ee.FeatureCollection(Map.draw_features)
roi_plume

In [58]:
# Get median of values in this polygon using ee.Image.reduceRegion
plume_enhancement_median = ee.Image.reduceRegion(image = area_10km, reducer = ee.Reducer.median(), geometry = roi_plume, scale = 20)
plume_enhancement_median

#Calculate sum of plume enhancement (~ -7.65 according to one go... take the absolute of this since everything is negative)

plume_enhancement_sum  = ee.Image.reduceRegion(image = area_10km, reducer = ee.Reducer.sum(), geometry = roi_plume, scale = 20)
plume_enhancement_sum.getInfo()

# Getting IME: 1) Convert enhancment map values from mol/m2 to kg/m2

# Get enhancement map of just plume by clipping to its geometry
roi_plume_geom = roi_plume.geometry()
plume_only_map = enhancement_MBMP.clip(roi_plume_geom)

# Convert plume map values from mol/m2 to kg/m2
plume_values_kgm2 = plume_only_map.multiply(16.4/1000)


# Get IME for each pixel and across whole area
IME_pixel = plume_values_kgm2.multiply(400)


# Get plume IME - Calculate sum of plume enhancement 
IME  = abs(ee.Image.reduceRegion(image = IME_pixel, reducer = ee.Reducer.sum(), geometry = roi_plume, scale = 20).get('constant').getInfo())
IME # in kg

# Get plume area in m^2 as a float
plume_area = roi_plume_geom.area(maxError=1).getInfo()
plume_area

L = np.sqrt(plume_area)
L

# Q in kg/s = (IME*\ueff)/l 
Q_kgs = (IME * 3)/L
Q_kgs

# Q in t/hour (3600s in 1 hour, 1000kg in 1t)
Q_th = Q_kgs*3600/1000
Q_th

2.168610949627769

In [60]:
print('IME :', IME, ' \nQ in kg/s:', Q_kgs, '\nQ in t/hour:', Q_th )

IME : 49.723522277787005  
Q in kg/s: 0.6023919304521581 
Q in t/hour: 2.168610949627769


## Cut cells

#### 04/03/24 - what is my plot showing? Can I extract the enhancement?

In [17]:
m = geemap.Map()
multi_band_mask_img = img.mask()
display('Multi-band mask image', multi_band_mask_img)
m.add_layer(multi_band_mask_img, None, 'Multi-band mask image')

single_band_mask_img = img.select('B12').mask()
display('Single-band mask image', single_band_mask_img)
m.add_layer(single_band_mask_img, None, 'Single-band mask image')
m

'Multi-band mask image'

'Single-band mask image'

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

#### Get median enhancement outside plume

In [ ]:
# test
surr = np.where(geometry_area.contains(roi_plume) == False)

surr

In [84]:
# Subtract plume area from clipped region

Surrounding_area = area_10km.subtract(roi_plume)

In [ ]:
Map = geemap.Map()

# Clip MBMP image to the geometry area of fig 2f in Varon 2021

vis = {'min': -0.2, 'max' : 0.2, "palette": ['FF0000',	'FFFFFF', '0000FF']}
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre
Map.addLayer(Surrounding_area, vis, 'Clipped Image')
Map.add_colorbar(vis, label='Methane enhancement ')
Map

In [62]:
Map = geemap.Map()
Map.addLayer(roi_not_plume)
Map.setCenter(source_lon, source_lat, 13.5) # Define map centre

Map

Map(center=[45.3324, 52.373], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [37]:
# Compute median value for area out of this region by masking 
# Use https://developers.google.com/earth-engine/tutorials/tutorial_api_05

roi_not_plume = Hassi_Messaoud_area.clip(roi_plume)
roi_not_plume

In [35]:
surroundings_median = ee.Image.reduceRegion(image = Hassi_Messaoud_area, reducer = ee.Reducer.median(), geometry = roi_not_plume, scale = 20)
surroundings_median

In [81]:
# Try setting plume values to NaN

polygon_contains = geometry_area.contains(roi_plume)

In [82]:
polygon_contains

In [83]:
ig = area_10km[area_10km >= 0]

TypeError: '>=' not supported between instances of 'Image' and 'int'

In [ ]:
# Try subtractng one image from the other


In [ ]:
# median of area_10km

area_10km_median = ee.Image.reduceRegion(image = area_10km, reducer = ee.Reducer.median(), geometry = geometry_area, scale = 20)

area_10km_median


#### plume q

In [151]:
roi_plume_geom = roi_plume.geometry()
roi_plume_geom

# Get plume area in m^2 as a float
plume_area = roi_plume_geom.area(maxError=1).getInfo()
plume_area

L = np.sqrt(plume_area)
L

213.59790463409877

In [152]:
# Get plume area in hectares (not sure we should use this for source rate since that is in kg/s)
plume_area_hectares = plume_area*0.0001
plume_area_hectares

4.562406486407756

In [119]:
L_hectares = np.sqrt(plume_area_hectares)
L_hectares

2.179752067708664

In [138]:
sum = abs(plume_enhancement_sum.get('constant').getInfo())

In [139]:
IME = sum*(plume_area
IME

363635.92677710555

In [142]:
# Q = (IME*\ueff)/l
Q = (363636 * 10)/L
Q


16682.447760320323

#### (cut from hassi messaoud section)Get median values inside and outside plume by drawing it

In [18]:
#Define region of interest as plume polygon drawn above
roi_plume = ee.FeatureCollection(Map.draw_features)
roi_plume

In [19]:
# Compute median value for area out of this region by masking 
# Use https://developers.google.com/earth-engine/tutorials/tutorial_api_05

roi_not_plume = Hassi_Messaoud_area.clip(roi_plume)
roi_not_plume

In [20]:
# Get median of values in this polygon using ee.Image.reduceRegion
plume_enhancement_median = ee.Image.reduceRegion(image = Hassi_Messaoud_area, reducer = ee.Reducer.median(), geometry = roi, scale = 20)
plume_enhancement_median

NameError: name 'roi' is not defined

In [ ]:
surroundings_median = ee.Image.reduceRegion(image = Hassi_Messaoud_area, reducer = ee.Reducer.median(), geometry = roi_not_plume, scale = 20)
print(surroundings_median)

In [22]:
# Using method from https://developers.google.com/earth-engine/guides/reducers_reduce_region
# except that there is only one band here, so we are getting mean rather than mean dictionary

mean = F_MBSP.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

# print(mean.get('B11').getInfo())
mean.getInfo()

{'constant': -0.14809884283750505}

In [23]:
# Test with B12 only to see if getting same result

B12 = img.select('B12') # Get B12 only out of image
B12

meanb12 = B12.reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

meanb12.getInfo()


{'B12': 5449.952903610025}

In [24]:
# Test with B12 only to see if getting same result

B12 = img.select('B12') # Get B12 only out of image
B12

maxb12 = B12.reduceRegion(
    reducer = ee.Reducer.max(),
    geometry = geometry_area,
    scale = 30,
    maxPixels = 1e9
)

maxb12.getInfo()


{'B12': 6290}

In [26]:
# Plot same as above but with empirical scaling factor c = 0.9475287960061192 (from least squares fitting calculation above)

# Basemap
Map = geemap.Map() # plot basemap

R_MBSP = img.expression(
    '(((0.9475287960061192 * R12) - R11)/R11 )', # Where this number comes from least squares difference scale above
    {
        'R11': img.select('B11').divide(10000),
        'R12': img.select('B12').divide(10000),
        # 'R11': img.select('B11'),
        # 'R12': img.select('B12')
    },
)

# Methane column enhancement
F_MBSP = R_MBSP.subtract(-0.029)

# Define visualisation parameters for plot


vis = {'min': -0.38, 'max': 0.04, "palette": ['FF0000',	'FFFFFF', '0000FF']	}


# Add data layers to basemap and plot
Map.setCenter(source_lon, source_lat, 13.75) # Define map centre and zoom
Map.addLayer(F_MBSP, vis, 'Sentinel-2') # Add bands of Sentinel-2 data
# Map.addLayer(geometry_area, {}, 'geometry_area') # Overlay rectangle used in Varon (2021) figure 2e
Map.add_colorbar(vis, label='Methane enhancement ')
Map

Map(center=[31.6585, 5.9053], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…